<a href="https://colab.research.google.com/github/Vurimindi2021/Capstone3/blob/main/src/nlpaug2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

In [ ]:
#nlp augmentation
!pip install -U deep_translator
!pip install detectlanguage
!pip install dask[Bag]
!pip install numpy requests nlpaug
!pip install torch>=1.2.0 transformers>=2.0.0
!pip install nltk>=3.4.5
!pip install transformers
!pip install --quiet googletrans
!pip install dask[Bag]

In [ ]:
#for fast parallel processing
from dask import bag, diagnostics
#python basics
import math, os, re, time
import numpy as np
import pandas as pd
#nlp augumentation
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
from nlpaug.util import Action
import transformers
from transformers import BertTokenizer, TFBertModel
from transformers import AutoTokenizer
import torch
import detectlanguage
from deep_translator import (GoogleTranslator,
                             MicrosoftTranslator,
                             PonsTranslator,
                             LingueeTranslator,
                             MyMemoryTranslator,
                             YandexTranslator,
                             PapagoTranslator,
                             DeepL,
                             QCRI,
                             single_detection,
                             batch_detection)
detectlanguage.configuration.api_key = "7e2a33b9ece836bec4d352be16ad4412"

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/nlp/data/train.csv")
test = pd.read_csv("/content/drive/MyDrive/nlp/data/test.csv")
print("Reading input data is done")
df_train=train.copy()

In [ ]:
#Instantiating Augumentaion Function
aug = naw.ContextualWordEmbsAug(model_path='bert-base-multilingual-uncased', aug_p=0.1)
print("Apply Augumentation Function")

In [ ]:
#Apply Augumentation Function
df_train['aug_premise']= df_train['premise'].apply(lambda x: aug.augment(x))
print("Apply ContextualWordEmbsAug Function Done")

In [ ]:
len(df_train)

In [ ]:
len(df_train[df_train['aug_premise'].str.contains("UNK")])

In [ ]:
df_train.drop(df_train.loc[df_train['aug_premise'].str.contains("UNK")].index, inplace=True)

In [ ]:
len(df_train)

In [ ]:
df_train.to_csv("/content/drive/MyDrive/nlp/data/contextaug_train.csv")

In [ ]:
df_train=pd.read_csv("/content/drive/MyDrive/nlp/data/contextaug_train.csv")

In [ ]:
len(df_train)

In [ ]:
df_train.head(5)

In [ ]:
def translate(sequence1, sequence2):
  langlist = ['en', 'fr', 'th', 'tr', 'ur', 'ru', 'bg', 'de', 'ar', 'zh', 'hi','sw', 'vi', 'es', 'el']
  langdict =  {'aa':'Afar','ab':'Abkhazian','af':'Afrikaans','ak':'Akan','am':'Amharic','ar':'Arabic','as':'Assamese',
         'ay':'Aymara','az':'Azerbaijani','ba':'Bashkir','be':'Belarusian','bg':'Bulgarian','bh':'Bihari',
         'bi':'Bislama','bn':'Bengali','bo':'Tibetan','br':'Breton','bs':'Bosnian','bug':'Buginese','ca':'Catalan',
         'ceb':'Cebuano','chr':'Cherokee','co':'Corsican','crs':'Seselwa','cs':'Czech','cy':'Welsh','da':'Danish',
         'de':'German','dv':'Dhivehi','dz':'Dzongkha','egy':'Egyptian','el':'Greek','en':'English','eo':'Esperanto',
         'es':'Spanish','et':'Estonian','eu':'Basque','fa':'Persian','fi':'Finnish','fj':'Fijian','fo':'Faroese',
         'fr':'French','fy':'Frisian','ga':'Irish','gd':'ScotsGaelic','gl':'Galician','gn':'Guarani','got':'Gothic',
         'gu':'Gujarati','gv':'Manx','ha':'Hausa','haw':'Hawaiian','hi':'Hindi','hmn':'Hmong','hr':'Croatian',
         'ht':'HaitianCreole','hu':'Hungarian','hy':'Armenian','ia':'Interlingua','id':'Indonesian','ie':'Interlingue',
         'ig':'Igbo','ik':'Inupiak','is':'Icelandic','it':'Italian','iu':'Inuktitut','iw':'Hebrew', 'ja':'Japanese',
         'jw':'Javanese','ka':'Georgian','kha':'Khasi','kk':'Kazakh','kl':'Greenlandic','km':'Khmer','kn':'Kannada',
         'ko':'Korean','ks':'Kashmiri','ku':'Kurdish','ky':'Kyrgyz','la':'Latin','lb':'Luxembourgish','lg':'Ganda',
         'lif':'Limbu','ln':'Lingala','lo':'Laothian','lt':'Lithuanian','lv':'Latvian','mfe':'MauritianCreole',
         'mg':'Malagasy','mi':'Maori','mk':'Macedonian','ml':'Malayalam','mn':'Mongolian','mr':'Marathi','ms':'Malay',
         'mt':'Maltese','my':'Burmese','na':'Nauru','ne':'Nepali','nl':'Dutch','no':'Norwegian','nr':'Ndebele',
         'nso':'Pedi','ny':'Nyanja','oc':'Occitan','om':'Oromo','or':'Oriya','pa':'Punjabi','pl':'Polish',
         'ps':'Pashto','pt':'Portuguese','qu':'Quechua','rm':'RhaetoRomance','rn':'Rundi','ro':'Romanian',
         'ru':'Russian','rw':'Kinyarwanda','sa':'Sanskrit','sco':'Scots','sd':'Sindhi','sg':'Sango','si':'Sinhalese',
         'sk':'Slovak','sl':'Slovenian','sm':'Samoan','sn':'Shona','so':'Somali','sq':'Albanian','sr':'Serbian',
         'ss':'Siswant','st':'Sesotho','su':'Sundanese','sv':'Swedish','sw':'Swahili','syr':'Syriac','ta':'Tamil',
         'te':'Telugu','tg':'Tajik','th':'Thai','ti':'Tigrinya','tk':'Turkmen','tl':'Tagalog','tlh':'Klingon',
         'tn':'Tswana','to':'Tonga','tr':'Turkish','ts':'Tsonga','tt':'Tatar','ug':'Uighur','uk':'Ukrainian',
         'ur':'Urdu','uz':'Uzbek','ve':'Venda','vi':'Vietnamese','vo':'Volapuk','war':'WarayPhilippines',
         'wo':'Wolof','xh':'Xhosa','yi':'Yiddish','yo':'Yoruba','za':'Zhuang','zh':'ChineseSimplified',
         'zh-Hant':'ChineseTraditional','zu':'Zulu'}
  #store original language so we can convert back
  org_lang =detectlanguage.simple_detect(sequence1)
  #randomly choose language to translate sequence to
  random_lang = np.random.choice([lang for lang in langlist if lang is not org_lang])
  if org_lang in langlist:
    translated1 = GoogleTranslator(source='auto', target=random_lang).translate(sequence1)
    translated2 = GoogleTranslator(source='auto', target=random_lang).translate(sequence2)
    output_sequence1 = translated1
    output_sequence2 = translated2
    langabv          = random_lang      
    languagefull     = langdict[random_lang]

    #if detected language not in our list of languages, do nothing
  else:
    output_sequence1 = sequence1
    output_sequence2 = sequence2
    langabv          = org_lang
    languagefull     = langdict[org_lang]
  return output_sequence1, output_sequence2,langabv, languagefull

In [ ]:
#applies above define function with Dask
def translate_parallel(dataset):
  with diagnostics.ProgressBar():
    #pair premises and hypothesis
    dataset[['premise', 'hypothesis', 'lang_abv', 'language']] = \
    dataset[['aug_premise', 'hypothesis']].apply(lambda x: pd.Series(translate(x.aug_premise, x.hypothesis)),axis=1)
  return dataset

In [ ]:
translate_parallel(df_train)

In [ ]:
df_train.id=df_train.id.apply(lambda x: 't'+ x)

In [ ]:
landict={'aa':'Afar','ab':'Abkhazian','af':'Afrikaans','ak':'Akan','am':'Amharic','ar':'Arabic','as':'Assamese',
         'ay':'Aymara','az':'Azerbaijani','ba':'Bashkir','be':'Belarusian','bg':'Bulgarian','bh':'Bihari',
         'bi':'Bislama','bn':'Bengali','bo':'Tibetan','br':'Breton','bs':'Bosnian','bug':'Buginese','ca':'Catalan',
         'ceb':'Cebuano','chr':'Cherokee','co':'Corsican','crs':'Seselwa','cs':'Czech','cy':'Welsh','da':'Danish',
         'de':'German','dv':'Dhivehi','dz':'Dzongkha','egy':'Egyptian','el':'Greek','en':'English','eo':'Esperanto',
         'es':'Spanish','et':'Estonian','eu':'Basque','fa':'Persian','fi':'Finnish','fj':'Fijian','fo':'Faroese',
         'fr':'French','fy':'Frisian','ga':'Irish','gd':'ScotsGaelic','gl':'Galician','gn':'Guarani','got':'Gothic',
         'gu':'Gujarati','gv':'Manx','ha':'Hausa','haw':'Hawaiian','hi':'Hindi','hmn':'Hmong','hr':'Croatian',
         'ht':'HaitianCreole','hu':'Hungarian','hy':'Armenian','ia':'Interlingua','id':'Indonesian','ie':'Interlingue',
         'ig':'Igbo','ik':'Inupiak','is':'Icelandic','it':'Italian','iu':'Inuktitut','iw':'Hebrew', 'ja':'Japanese',
         'jw':'Javanese','ka':'Georgian','kha':'Khasi','kk':'Kazakh','kl':'Greenlandic','km':'Khmer','kn':'Kannada',
         'ko':'Korean','ks':'Kashmiri','ku':'Kurdish','ky':'Kyrgyz','la':'Latin','lb':'Luxembourgish','lg':'Ganda',
         'lif':'Limbu','ln':'Lingala','lo':'Laothian','lt':'Lithuanian','lv':'Latvian','mfe':'MauritianCreole',
         'mg':'Malagasy','mi':'Maori','mk':'Macedonian','ml':'Malayalam','mn':'Mongolian','mr':'Marathi','ms':'Malay',
         'mt':'Maltese','my':'Burmese','na':'Nauru','ne':'Nepali','nl':'Dutch','no':'Norwegian','nr':'Ndebele',
         'nso':'Pedi','ny':'Nyanja','oc':'Occitan','om':'Oromo','or':'Oriya','pa':'Punjabi','pl':'Polish',
         'ps':'Pashto','pt':'Portuguese','qu':'Quechua','rm':'RhaetoRomance','rn':'Rundi','ro':'Romanian',
         'ru':'Russian','rw':'Kinyarwanda','sa':'Sanskrit','sco':'Scots','sd':'Sindhi','sg':'Sango','si':'Sinhalese',
         'sk':'Slovak','sl':'Slovenian','sm':'Samoan','sn':'Shona','so':'Somali','sq':'Albanian','sr':'Serbian',
         'ss':'Siswant','st':'Sesotho','su':'Sundanese','sv':'Swedish','sw':'Swahili','syr':'Syriac','ta':'Tamil',
         'te':'Telugu','tg':'Tajik','th':'Thai','ti':'Tigrinya','tk':'Turkmen','tl':'Tagalog','tlh':'Klingon',
         'tn':'Tswana','to':'Tonga','tr':'Turkish','ts':'Tsonga','tt':'Tatar','ug':'Uighur','uk':'Ukrainian',
         'ur':'Urdu','uz':'Uzbek','ve':'Venda','vi':'Vietnamese','vo':'Volapuk','war':'WarayPhilippines',
         'wo':'Wolof','xh':'Xhosa','yi':'Yiddish','yo':'Yoruba','za':'Zhuang','zh':'ChineseSimplified',
         'zh-Hant':'ChineseTraditional','zu':'Zulu'}

In [ ]:
lan=['aa','ab','af','ak','am','ar','as','ay','az','ba','be','bg','bh','bi','bn','bo','br','bs','bug','ca',
               'ceb','chr','co','crs','cs','cy','da','de','dv','dz','egy','el','en','eo','es','et','eu','fa','fi',
               'fj','fo','fr','fy','ga','gd','gl','gn','got','gu','gv','ha','haw','hi','hmn','hr','ht','hu','hy','ia',
               'id','ie','ig','ik','is','it','iu','iw','ja','jw','ka','kha','kk','kl','km','kn','ko','ks','ku','ky',
               'la','lb','lg','lif','ln','lo','lt','lv','mfe','mg','mi','mk','ml','mn','mr','ms','mt','my','na','ne',
               'nl','no','nr','nso','ny','oc','om','or','pa','pl','ps','pt','qu','rm','rn','ro','ru','rw','sa','sco',
               'sd','sg','si','sk','sl','sm','sn','so','sq','sr','ss','st','su','sv','sw','syr','ta','te','tg','th',
               'ti','tk','tl','tlh','tn','to','tr','ts','tt','ug','uk','ur','uz','ve','vi','vo','war','wo','xh','yi',
               'yo','za','zh','zh-Hant','zu']

In [ ]:
set('en':'English','fr':'French', 'th':'Thai', 'ur':'Urdu', 'ru':'Russian', 'zh':'Chinese',
               'hi':'Hindi','sw':'Swahili', 'vi':'Vietnamese', 'es':'Spanish', 'el':'Greek', 

In [ ]:
df_train.to_csv("/content/drive/MyDrive/nlp/data/aug_train.csv")